In [ ]:
# Load required libraries

import zipfile
import numpy as np
import pandas as pd
# import pdfplumber
from dotenv import load_dotenv
import os
# import pytesseract as pyt
import fitz
import pymupdf
# from pdf2image import convert_from_path
# from PIL import Image, ImageEnhance, ImageFilter
# import cv2
# from tesserocr import PyTessBaseAPI, RIL
# import tesserocr
import json
import pprint
import re
from dotenv import find_dotenv
from dotenv import load_dotenv

# marker functions
from marker.converters.ocr import OCRConverter
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.config.parser import ConfigParser

In [ ]:
import torch
torch.cuda.is_available()
torch.zeros(1).cuda()

In [ ]:
testfile = "page1-cli-ocr-force-strip-chars"
directory = "/content/testing/" + testfile + ".pdf"

config = {
    "output_format": "json",
    "OCRJSONRenderer_extract_images": "True",
    "disable_image_extraction": "True",
    "DEBUG":"True",
    "force_ocr":"True",
    "strip_existing_ocr": "True"
}
config_parser = ConfigParser(config)
converter = OCRConverter(
    config=config_parser.generate_config_dict(),
    artifact_dict=create_model_dict(),
)
rendered = converter(directory)

output_filename = "/content/testing/" + "marker-" + testfile + ".json"
with open(output_filename, "w", encoding="utf-8") as f:
    f.write(rendered.model_dump_json())

In [ ]:
# Test if individual file is repaired (before uploading to marker-pdf)
# quickcheckname = "textract_test"
# doc = pymupdf.open("./docs_to_write_on/"+ quickcheckname + ".pdf")
# print(doc.can_save_incrementally())
# doc.close()
# page=None

## Marker testing

In [ ]:
# Read file-list to process

pdfolder = "./docs_to_write_on/"
jsonfolder = "./jsons_to_read/"
nameslist = set([filename[:-4] for filename in os.listdir(pdfolder)])   # read all files in directory and cut off extensions (keep file names only), set-type object for faster comparison later
print(nameslist)

# use this portion to opt in/out files to be run later

opt_out = {'page1test'}
nameslist = [name for name in nameslist if name not in opt_out]
# print(nameslist)   

#replace cutting off 'marker-' with regex
verificationset = set([filename[7:] for filename in [filename[:-5] for filename in os.listdir(jsonfolder)]]).symmetric_difference(nameslist)   # cut off marker- prefix and .json extension from json files and compare to namelist sets to verify no issues
if verificationset == opt_out:
    verificationset = set()
print(verificationset)

In [ ]:
# Verify files before uploading to marker or opening with PyMuPDF

def check_ok_for_marker(filename, filedir):
    
    init_file = filedir + filename + ".pdf"
    doc = pymupdf.open(init_file)

    if not doc.can_save_incrementally():

        print(f'File: "{filename}" unwritable. Fixing...')

        new_file = filedir + filename + "-fixed" + ".pdf"
        doc.save(new_file, garbage=4,deflate=True)
        doc.close()
        page=None
        os.remove(init_file)
        os.rename(new_file,init_file)
    else:
        print('Nothing to fix!')

if len(verificationset)==0 :   # change this to check only files that are outside the verificationlist
    for pdf_file in nameslist:
        check_ok_for_marker(pdf_file,pdfolder)

In [ ]:
renderedlist = []
for json_file in nameslist:
    with open(jsonfolder + "marker-" + json_file + ".json", "r", encoding="utf-8") as f:
        renderedlist.append(json.load(f)['children'])

# for json_file in renderedlist:
#   pprint.pprint(json_file, indent=2)
print(renderedlist)

In [ ]:
# Retrieve specific defined coordinates from JSON, "flatten" data

# Marker-defined categories (specifically leaves only) to extract from marker output JSON. Parse through the output JSON to find new leaf categories to add for extraction
leaf_categories_with_text = ['Line','Text','TextInlineMath','Caption','ListItem','SectionHeader','TableCell', 'PageFooter','PageHeader']
leaf_categories_to_retrieve = ['Picture','TableGroup','Equation', 'Figure', 'Handwriting']

# For data retrieved from parsing with the full marker PDF model. Recursive function to retrieve all coordinates and text of identified objects
def retrieve_specific_boxes_and_flatten(treelist,pull_polygons,pull_text):
  # recursive function to pick up all the coordinates, and all HTML text from specifically defined configurations of nested items
  coordinates = {}
  htmtext = {}
  
  for nitem,item in enumerate(treelist):
    newpoly = {}
    newtext = {}
    if (item['children']):
      # newpoly = {'size':[tuple(point) for point in item['polygon']]}
      newchildren = item['children']
      retr_coords,retr_text = retrieve_specific_boxes_and_flatten(newchildren,pull_polygons,pull_text)
      newpoly.update(retr_coords)
      newtext.update(retr_text)
      if item['block_type']=='Page':
        newitem_poly = {int(re.search(r"/page/(\d+)/", item['id']).group(1)):newpoly}
        newitem_text = {int(re.search(r"/page/(\d+)/", item['id']).group(1)):newtext}
      else: 
        newitem_poly = newpoly
        newitem_text = newtext
    else:
      if (item['block_type'] in pull_polygons) or (item['block_type'] in pull_text) :
        newpoly = item['polygon']
        newpoly = [tuple(point) for point in newpoly]
        newitem_poly = {item['id']:newpoly}
        if item['block_type'] in pull_text:
          newtext = item['html']
          newitem_text = {item['id']:newtext}
        else:
          newitem_text = {}
      else: 
        newitem_poly = {}
        newitem_text = {}
    coordinates.update(newitem_poly)
    htmtext.update(newitem_text)
  return (coordinates,htmtext)

# For data retrieved from parsing with the OCR-ONLY marker PDF model with the --keep-chars flag active. Recursive function to retrieve all coordinates and 
# text of identified objects. The difference from the full model function are the 'Char' objects. They do not have a 'children' attribute and the retrieval throws an error
# (could technically only use this function but I like to have them differentiated)
def ocr_retrieve_specific_boxes_and_flatten(treelist,pull_polygons,pull_text):
  # recursive function to pick up all the coordinates from specifically defined configurations of nested items
  coordinates = {}
  htmtext = {}
  
  for nitem,item in enumerate(treelist):
    newpoly = {}
    newtext = {}
    if (item['block_type'] == 'Char'):
        newpoly = item['polygon']
        newpoly = [tuple(point) for point in newpoly]
        newitem_poly = {item['id']:newpoly}
        newtext = item['text']
        newitem_text = {item['id']:newtext}
    else:
        if (item['children']):
            # newpoly = {'size':[tuple(point) for point in item['polygon']]}
            newchildren = item['children']
            retr_coords,retr_text = ocr_retrieve_specific_boxes_and_flatten(newchildren,pull_polygons,pull_text)
            newpoly.update(retr_coords)
            newtext.update(retr_text)
            if item['block_type']=='Page':
                newitem_poly = {int(re.search(r"/page/(\d+)/", item['id']).group(1)):newpoly}
                newitem_text = {int(re.search(r"/page/(\d+)/", item['id']).group(1)):newtext}
            else: 
                newitem_poly = newpoly
                newitem_text = newtext
        else:
            if (item['block_type'] in pull_polygons) or (item['block_type'] in pull_text) :
                newpoly = item['polygon']
                newpoly = [tuple(point) for point in newpoly]
                newitem_poly = {item['id']:newpoly}
                if item['block_type'] in pull_text:
                    newtext = item['html']
                    newitem_text = {item['id']:newtext}
                else:
                    newitem_text = {}
            else: 
                newitem_poly = {}
                newitem_text = {}
    coordinates.update(newitem_poly)
    htmtext.update(newitem_text)
  return (coordinates,htmtext)



renderedboxes = []
renderedtext = []

for name,red_lines in zip(nameslist,renderedlist):
  if re.search('(chars)', name):
    all_boxes,all_text = ocr_retrieve_specific_boxes_and_flatten(red_lines,leaf_categories_to_retrieve,leaf_categories_with_text)
  else:
    all_boxes,all_text = retrieve_specific_boxes_and_flatten(red_lines,leaf_categories_to_retrieve,leaf_categories_with_text)


  renderedboxes.append(all_boxes)
  renderedtext.append(all_text) 

for boxes,texts in zip(renderedboxes,renderedtext):
   display(boxes)
   display(texts)



###############################################################################################################

# old JSON processing function, workd for simple OCR model, fails for Full model

# def retrieve_all_boxes(treelist):
#   # recursive function to pick up all the coordinates from any configuration of nested items
#   newdict = {}
#   # location = location + " " + str(nitem)
#   # all_boxes.update({location:item['polygon']})
#   for nitem,item in enumerate(treelist):
#     if (item['children']):
#       newpoly = {'size':[tuple(point) for point in item['polygon']]}
#       newchildren = item['children']
#       newpoly.update(retrieve_all_boxes(newchildren))
#       if item['block_type']=='Page':
#         newid = int(re.search(r"/page/(\d+)/", item['id']).group(1))
#       else: newid = item['id']
#     else:
#       newpoly = item['polygon']
#       newpoly = [tuple(point) for point in newpoly]
#       newid = item['id']
   
#     newdict.update({newid:newpoly})
#   return newdict

# all_boxes = retrieve_all_boxes(red_lines)
# display(all_boxes)

In [ ]:
# write coordinate boxes to PDF


def add_boxes(filename, filedir, annotations):

  # open current document
  init_file = filedir + filename + ".pdf"
  doc = pymupdf.open(init_file)

  # check file health and save as new file (fixes marker-pdf PIL errors and PyMuPDF 'code=4' errors)
  if not doc.can_save_incrementally():
    print(f'File: "{filename}" unwritable. Fixing...')
    # filename = filename + "-fixed"
    new_file = filedir + filename + "-fixed" + ".pdf"
    doc.save(new_file, garbage=4,deflate=True)
    doc.close()
    page=None
    os.remove(init_file)
    os.rename(new_file,init_file)
    doc = pymupdf.open(init_file)
  
  for i,page in enumerate(doc):
    lines = annotations[i]
    nannots = len(lines)
    for ncoord,coords in enumerate(lines.items()):
      print(f"Page {i}: Adding annotation {ncoord} / {nannots}",end='\r')
      annot = page.add_polygon_annot(coords[1])
      annot.set_colors(stroke=(0.416, 0.416, 1))
      annot.update()
      doc.saveIncr()
  doc.close()
  page=None
  print(f'"{filename}" annotated.')


def print_retrieved_text(texttoprint):
  for item in texttoprint.values():
    for textblock in item.values():
      print(re.sub( "<(\w+)([\s\-\w='\":/\.]+)?>" ,"",re.sub( "<(/\w+)([\s\w=':/\.]+)?>" ,"",textblock)))

for name,boxes,texts in zip(nameslist,renderedboxes,renderedtext):
  add_boxes(name, pdfolder, boxes)
  print_retrieved_text(texts)

-----

## Kofax: Pulling signatures from processed PDF 
(using Kofax, transform to Word and then back to PDF)

In [ ]:
import re, fitz, unicodedata

# ---- helpers ---- 
def norm(s):
    s = unicodedata.normalize('NFKC', str(s))
    return s.replace('\u00B7','.')  # middle dot → '.'

# τίτλοι που ενεργοποιούν Wipe-Βand
HEAD_TRIGGERS = [
  'ΟΙ ΣΥΜΒΑΛΛΟΜΕΝΟΙ', 'Ο ΣΥΜΒΑΛΛΟΜΕΝΟΣ', 'Η ΣΥΜΒΑΛΛΟΜΕΝΗ',
  'Ο ΣΥΝΕΤΑΙΡΟΣ ΠΙΣΤΟΥΧΟΣ', 'ΟΙ ΣΥΝΟΦΕΙΛΕΤΗΣ', 'Ο/ΟΙ ΣΥΝΟΦΕΙΛΕΤΗΣ/ΕΣ',
  'Ο/ΟΙ ΕΓΓΥΗΤΗΣ/ΕΣ', 'Ο ΕΓΓΥΗΤΗΣ', 'Ο ΟΦΕΙΛΕΤΗΣ', 'Ο/ΟΙ ΟΦΕΙΛΕΤΗΣ/ΕΣ'
]

def text_blocks(page):
    # rawdict → blocks (text/images) με bbox
    data = page.get_text('rawdict')
    return data.get('blocks', []) if isinstance(data, dict) else []

def find_heading_band(page, side_margin=10, top_offset=6):
    # ψάξε αν υπάρχει τίτλος-κλειδί στη σελίδα και γύρνα ζώνη (x0,y0,x1,y1)
    W,H = page.rect.width, page.rect.height
    best_y = None
    for b in text_blocks(page):
        if b.get('type') != 0: continue  # μόνο text blocks
        for l in b.get('lines', []):
            line_text = ' '.join([s.get('text','') for s in l.get('spans',[])])
            T = norm(line_text).upper().strip()
            for key in HEAD_TRIGGERS:
                if key in T:
                    # πάρε bbox της γραμμής
                    xs=[]; ys=[]
                    for s in l.get('spans', []):
                        (x0,y0,x1,y1) = s.get('bbox', (None,None,None,None))
                        if x0 is None: continue
                        xs += [x0,x1]; ys += [y0,y1]
                    if xs and ys:
                        y_line = max(ys)  # κάτω άκρη γραμμής
                        best_y = y_line if best_y is None else min(best_y, y_line)
    if best_y is None: return None
    x0 = page.rect.x0 + side_margin
    y0 = best_y + top_offset
    x1 = page.rect.x1 - side_margin
    y1 = page.rect.y1 - 8
    return (x0,y0,x1,y1) if y0 < y1 else None

def image_boxes_in_band(page, band=None, bottom_ratio=0.35):
    boxes = []
    W,H = page.rect.width, page.rect.height
    bottom_y = H*(1.0-bottom_ratio)
    for b in text_blocks(page):
        if b.get('type') == 1:  # image
            (x0,y0,x1,y1) = b.get('bbox', (None,None,None,None))
            if x0 is None: continue
            # κριτήρια: (i) τέμνει band ή (ii) βρίσκεται στο κάτω Χ% της σελίδας
            in_bottom = (y0 >= bottom_y) or (y1 >= bottom_y)
            intersects_band = False
            if band is not None:
                bx0,by0,bx1,by1 = band
                intersects_band = not (x1<bx0 or x0>bx1 or y1<by0 or y0>by1)
            if intersects_band or in_bottom:
                boxes.append((x0,y0,x1,y1))
    return boxes

def redact_pdf(input_pdf, output_pdf, do_wipe=True, do_images=True, side_margin=10, top_offset=6, bottom_ratio=0.35, fill='white', draw_labels=False):
    rgb=(1,1,1) if str(fill).lower()=='white' else (0,0,0)
    doc = fitz.open(input_pdf)
    for pno in range(len(doc)):
        page = doc[pno]
        band = find_heading_band(page, side_margin=side_margin, top_offset=top_offset) if do_wipe else None
        if band:
            r = fitz.Rect(band)
            page.add_redact_annot(r, text=('[SIGN-BAND]' if draw_labels else None), fill=rgb)
        if do_images:
            for (x0,y0,x1,y1) in image_boxes_in_band(page, band=band, bottom_ratio=bottom_ratio):
                r = fitz.Rect(x0,y0,x1,y1)
                page.add_redact_annot(r, text=('[IMG]' if draw_labels else None), fill=rgb)
        page.apply_redactions()
    doc.save(output_pdf, garbage=4, deflate=True)
    doc.close()


redact_pdf('test_text_and_signaturesdoc.pdf',"out_signzones.pdf")

-----

## Azure: DocumentIntelligence for OCR

and integrating an NER model on the text

In [ ]:
load_dotenv(dotenv_path='./config/.env')

AZURE_ENDPOINT = os.getenv('AZURE_ENDPOINT')
AZURE_KEY = os.getenv('AZURE_KEY')

In [ ]:
AZURE_CONFIG = {
    "endpoint" : AZURE_ENDPOINT,
    "key" : AZURE_KEY
}

-----

## TesserOCR: Testing library to get coordinates

After pre-OCR-ing with Tungsten/Kofax PowerPDF

In [ ]:
# Standard Tesseract + output images (i think)

# Define Tesseract exe location
pyt.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
pyt.get_languages()

current_file = "testtest.pdf"



def ocr_scanned_pdf(pdf_path):
    text = ""
    images = convert_from_path(pdf_path)    
    for i, image in enumerate(images):
        page_text = pyt.image_to_string(image, lang="ell",config="--psm 4 --oem 3 -c tessedit_write_images=true")      
        print(f"OCR result for page {i+1}: {page_text}")
        # text += f"--- Page {i+1} ---\n{page_text}\n\n"
    return text


ocr_scanned_pdf(current_file)

In [ ]:
# Rewrite initial file to contain all coordinates of identified words


insertions = []

# open document
input_pdf = "testtest.pdf"
doc = fitz.open(input_pdf)

# insert here transform pdf to image
images = convert_from_path(input_pdf)
print(images)


def transform_coords(box):
    # assume of dict format with {'x':x,'y':y,'w':w,'h':h}
    x1 = box['x']
    y1 = box['y']
    w = box['w']
    h = box['h']
    x2 = x1+w
    y2 = y1+h

    # with polygon annotation:
    coordinates_pdf = [(x1*scale_x - 2,y1*scale_y - 2),(x2*scale_x + 2,y1*scale_y - 2), (x2*scale_x + 2,y2*scale_y + 2),(x1*scale_x - 2,y2*scale_y + 2)]

    # with redact annotation:


    return coordinates_pdf


for j,image in enumerate(images):

    insertions_temp_dict = {}

    with PyTessBaseAPI(path=r'C:\Program Files\Tesseract-OCR\tessdata', lang='ell') as api:


        api.SetImage(image)

        # get all sizes
        currentpage = j
        imgsize = image.size
        page = doc[currentpage]
        scale_x = page.rect.width / imgsize[0]
        scale_y = page.rect.height / imgsize[1]

        boxes = api.GetComponentImages(RIL.TEXTLINE, True)
        total_items = len(boxes)
        print('Found {} textline image components.'.format(total_items))
        for i, (im, box, _, _) in enumerate(boxes):

            api.SetRectangle(box['x'], box['y'], box['w'], box['h'])
            ocrResult = api.GetUTF8Text()
            conf = api.MeanTextConf()

            # create box visually on PDF and add embellishments
            pdf_transformed_coordinates = transform_coords(box)
            # print(pdf_transformed_coordinates)
            print(pdf_transformed_coordinates)
            annot = page.add_polygon_annot(pdf_transformed_coordinates)
            annot.set_colors(stroke=(0.416, 0.416, 1))
            annot.update()

            # # add text for visual representation
            # text_rect = fitz.Rect(pdf_transformed_coordinates[0][0],pdf_transformed_coordinates[0][1],pdf_transformed_coordinates[2][0],pdf_transformed_coordinates[2][1])
            # # text_rect = fitz.Rect(100,100,100,100)
            # page.insert_textbox(
            #     text_rect,
            #     ocrResult,
            #     fontsize=3,
            #     fontname="helv",
            #     color=(0, 0, 0),   # black text
            #     align=1            # center align
            # )

            # insertions_temp_dict.update({i:inserted})
            doc.saveIncr()
            print(f"\rProgress: {i+1}/{total_items}","\r",end="")
    insertions.append(insertions_temp_dict)




doc.close()
page = None

-----

## Untangling Char coords

In [ ]:
a= {'/page/0/Char/309': [(59.981864717805415, 287.13748905389093),
(59.981864717805415, 287.51234217276027),
(60.3567513722917, 287.51234217276027),
(60.3567513722917, 287.13748905389093)],
'/page/0/Char/310': [(59.981864717805415, 287.13748905389093),
(59.981864717805415, 287.51234217276027),
(60.3567513722917, 287.51234217276027),
(60.3567513722917, 287.13748905389093)],
'/page/0/Char/311': [(133.08476234263077, 288.2620484104989),
(154.82818830283523, 288.2620484104989),
(154.82818830283523, 293.1351389557999),
(133.08476234263077, 293.1351389557999)],
'/page/0/Char/312': [(140.58249543235644, 288.2620484104989),
(153.3286416848901, 288.2620484104989),
(153.3286416848901, 293.1351389557999),
(140.58249543235644, 293.1351389557999)],
'/page/0/Char/313': [(149.20488848554098, 288.2620484104989),
(159.32682815667062, 288.2620484104989),
(159.32682815667062, 293.1351389557999),
(149.20488848554098, 293.1351389557999)],
'/page/0/Char/314': [(149.20488848554098, 288.2620484104989),
(160.82637477461577, 288.2620484104989),
(160.82637477461577, 293.1351389557999),
(149.20488848554098, 293.1351389557999)],
'/page/0/Char/315': [(151.45420841245866, 288.2620484104989),
(160.0766014656432, 288.2620484104989),
(160.0766014656432, 293.5099920746692),
(151.45420841245866, 293.5099920746692)],
'/page/0/Char/316': [(156.32773492078036, 287.88719529162955),
(167.1994479008826, 287.88719529162955),
(167.1994479008826, 293.5099920746692),
(156.32773492078036, 293.5099920746692)],
'/page/0/Char/317': [(154.45330164834894, 287.88719529162955),
(165.69990128293745, 287.88719529162955),
(165.32501462845116, 293.5099920746692),
(154.07841499386265, 293.5099920746692)],
'/page/0/Char/318': [(157.4523948842392, 287.88719529162955),
(168.32410786434144, 287.88719529162955),
(168.32410786434144, 293.5099920746692),
(157.4523948842392, 293.5099920746692)],
'/page/0/Char/319': [(155.57796161180778, 288.2620484104989),
(163.82546801050603, 288.2620484104989),
(163.82546801050603, 293.1351389557999),
(155.57796161180778, 293.1351389557999)],
'/page/0/Char/320': [(152.9537550304038, 288.2620484104989),
(163.82546801050603, 288.2620484104989),
(163.82546801050603, 293.1351389557999),
(152.9537550304038, 293.1351389557999)],
'/page/0/Char/321': [(152.9537550304038, 288.2620484104989),
(162.7008080470472, 288.2620484104989),
(162.7008080470472, 293.5099920746692),
(152.9537550304038, 293.5099920746692)],
'/page/0/Char/322': [(153.70352833937636, 288.2620484104989),
(160.82637477461577, 288.2620484104989),
(160.82637477461577, 293.5099920746692),
(153.70352833937636, 293.5099920746692)],
'/page/0/Char/323': [(149.57977514002724, 288.2620484104989),
(159.7017148111569, 288.2620484104989),
(159.7017148111569, 293.5099920746692),
(149.57977514002724, 293.5099920746692)],
'/page/0/Char/324': [(149.20488848554098, 288.2620484104989),
(159.7017148111569, 288.2620484104989),
(159.7017148111569, 293.5099920746692),
(149.20488848554098, 293.5099920746692)],
'/page/0/Char/325': [(152.57886837591752, 288.2620484104989),
(162.3259213925609, 288.2620484104989),
(162.3259213925609, 293.1351389557999),
(152.57886837591752, 293.1351389557999)],
'/page/0/Char/326': [(147.70534186759582, 288.2620484104989),
(158.95194150218435, 288.2620484104989),
(158.95194150218435, 293.5099920746692),
(147.70534186759582, 293.5099920746692)],
'/page/0/Char/327': [(148.8300018310547, 288.2620484104989),
(157.4523948842392, 288.2620484104989),
(157.4523948842392, 293.5099920746692),
(148.8300018310547, 293.5099920746692)],
'/page/0/Char/328': [(147.70534186759582, 288.2620484104989),
(157.4523948842392, 288.2620484104989),
(157.4523948842392, 293.5099920746692),
(147.70534186759582, 293.5099920746692)],
'/page/0/Char/329': [(155.95284826629407, 288.2620484104989),
(165.32501462845116, 288.2620484104989),
(165.32501462845116, 293.8848451935385),
(155.95284826629407, 293.8848451935385)],
'/page/0/Char/330': [(154.82818830283523, 288.2620484104989),
(166.07478793742374, 288.2620484104989),
(166.07478793742374, 293.5099920746692),
(154.82818830283523, 293.5099920746692)],
'/page/0/Char/331': [(154.45330164834894, 288.2620484104989),
(160.82637477461577, 288.2620484104989),
(160.82637477461577, 293.5099920746692),
(154.45330164834894, 293.5099920746692)],
'/page/0/Char/332': [(155.20307495732152, 288.2620484104989),
(163.07569470153348, 288.2620484104989),
(163.07569470153348, 293.1351389557999),
(155.20307495732152, 293.1351389557999)],
'/page/0/Char/333': [(156.32773492078036, 288.2620484104989),
(166.07478793742374, 288.2620484104989),
(166.07478793742374, 293.5099920746692),
(156.32773492078036, 293.5099920746692)],
'/page/0/Char/334': [(159.32682815667062, 288.2620484104989),
(164.5752413194786, 288.2620484104989),
(164.5752413194786, 293.5099920746692),
(159.32682815667062, 293.5099920746692)],
'/page/0/Char/335': [(160.0766014656432, 288.2620484104989),
(164.5752413194786, 288.2620484104989),
(164.5752413194786, 293.5099920746692),
(160.0766014656432, 293.5099920746692)],
'/page/0/Char/336': [(160.0766014656432, 288.2620484104989),
(169.44876782780028, 288.2620484104989),
(169.44876782780028, 293.5099920746692),
(160.0766014656432, 293.5099920746692)],
'/page/0/Char/337': [(164.9501279739649, 287.88719529162955),
(169.82365448228657, 287.88719529162955),
(169.82365448228657, 293.5099920746692),
(164.9501279739649, 293.5099920746692)],
'/page/0/Char/338': [(167.1994479008826, 288.2620484104989),
(175.44695429958082, 287.88719529162955),
(175.44695429958082, 293.5099920746692),
(167.1994479008826, 293.5099920746692)],
'/page/0/Char/339': [(168.32410786434144, 288.2620484104989),
(177.69627422649853, 288.2620484104989),
(177.69627422649853, 293.5099920746692),
(168.32410786434144, 293.5099920746692)],
'/page/0/Char/340': [(171.3232011002317, 288.2620484104989),
(179.57070749892995, 288.2620484104989),
(179.57070749892995, 293.5099920746692),
(171.3232011002317, 293.5099920746692)],
'/page/0/Char/341': [(176.1967276085534, 288.2620484104989),
(181.07025411687508, 288.2620484104989),
(181.07025411687508, 293.5099920746692),
(176.1967276085534, 293.5099920746692)],
'/page/0/Char/342': [(177.69627422649853, 288.2620484104989),
(183.31957404379278, 288.2620484104989),
(183.31957404379278, 293.8848451935385),
(177.69627422649853, 293.8848451935385)],
'/page/0/Char/343': [(181.44514077136137, 288.2620484104989),
(187.06844058865562, 288.2620484104989),
(187.06844058865562, 293.5099920746692),
(181.44514077136137, 293.5099920746692)],
'/page/0/Char/344': [(182.56980073482023, 288.2620484104989),
(187.8182138976282, 288.2620484104989),
(187.8182138976282, 293.5099920746692),
(182.56980073482023, 293.5099920746692)],
'/page/0/Char/345': [(179.19582084444366, 288.2620484104989),
(187.8182138976282, 288.2620484104989),
(187.8182138976282, 293.5099920746692),
(179.19582084444366, 293.5099920746692)],
'/page/0/Char/346': [(181.44514077136137, 288.2620484104989),
(189.69264717005962, 288.2620484104989),
(189.69264717005962, 293.5099920746692),
(181.44514077136137, 293.5099920746692)],
'/page/0/Char/347': [(182.56980073482023, 288.2620484104989),
(190.4424204790322, 288.2620484104989),
(190.4424204790322, 293.5099920746692),
(182.56980073482023, 293.5099920746692)],
'/page/0/Char/348': [(179.94559415341624, 288.2620484104989),
(188.94287386108707, 288.2620484104989),
(188.94287386108707, 293.5099920746692),
(179.94559415341624, 293.5099920746692)],
'/page/0/Char/349': [(182.56980073482023, 288.2620484104989),
(188.1931005521145, 288.2620484104989),
(188.1931005521145, 293.5099920746692),
(182.56980073482023, 293.5099920746692)],
'/page/0/Char/350': [(186.69355393416936, 288.2620484104989),
(197.1903802597853, 288.2620484104989),
(197.1903802597853, 293.5099920746692),
(186.69355393416936, 293.5099920746692)],
'/page/0/Char/351': [(184.06934735276536, 288.2620484104989),
(192.6917404059499, 288.2620484104989),
(192.6917404059499, 293.5099920746692),
(184.06934735276536, 293.5099920746692)],
'/page/0/Char/352': [(185.94378062519678, 288.2620484104989),
(195.31594698735387, 288.2620484104989),
(195.31594698735387, 293.5099920746692),
(185.94378062519678, 293.5099920746692)],
'/page/0/Char/353': [(192.3168537514636, 288.2620484104989),
(202.063906768107, 288.2620484104989),
(202.063906768107, 293.5099920746692),
(192.3168537514636, 293.5099920746692)],
'/page/0/Char/354': [(197.94015356875786, 288.2620484104989),
(208.06209323988753, 288.2620484104989),
(208.06209323988753, 293.5099920746692),
(197.94015356875786, 293.5099920746692)],
'/page/0/Char/355': [(199.0648135322167, 288.2620484104989),
(206.5625466219424, 288.2620484104989),
(206.5625466219424, 293.5099920746692),
(199.0648135322167, 293.5099920746692)],
'/page/0/Char/356': [(200.93924680464812, 288.2620484104989),
(206.93743327642866, 288.2620484104989),
(206.93743327642866, 293.5099920746692),
(200.93924680464812, 293.5099920746692)],
'/page/0/Char/357': [(201.3141334591344, 288.2620484104989),
(208.81186654886008, 288.2620484104989),
(208.81186654886008, 293.5099920746692),
(201.3141334591344, 293.5099920746692)],
'/page/0/Char/358': [(198.31504022324415, 288.2620484104989),
(208.06209323988753, 288.2620484104989),
(208.06209323988753, 293.5099920746692),
(198.31504022324415, 293.5099920746692)],
'/page/0/Char/359': [(190.81730713351848, 288.2620484104989),
(200.56436015016186, 288.2620484104989),
(200.56436015016186, 293.5099920746692),
(190.81730713351848, 293.5099920746692)],
'/page/0/Char/360': [(186.69355393416936, 288.2620484104989),
(193.44151371492245, 288.2620484104989),
(193.44151371492245, 293.5099920746692),
(186.69355393416936, 293.5099920746692)],
'/page/0/Char/361': [(179.57070749892995, 288.2620484104989),
(188.56798720660078, 288.2620484104989),
(188.56798720660078, 293.5099920746692),
(179.57070749892995, 293.5099920746692)],
'/page/0/Char/362': [(191.94196709697732, 288.2620484104989),
(202.063906768107, 288.2620484104989),
(202.063906768107, 293.5099920746692),
(191.94196709697732, 293.5099920746692)],
'/page/0/Char/363': [(175.8218409540671, 288.2620484104989),
(188.1931005521145, 288.2620484104989),
(188.1931005521145, 293.5099920746692),
(175.8218409540671, 293.5099920746692)],
'/page/0/Char/364': [(174.69718099060827, 288.2620484104989),
(185.1940073162242, 288.2620484104989),
(185.1940073162242, 293.1351389557999),
(174.69718099060827, 293.1351389557999)],
'/page/0/Char/365': [(173.19763437266312, 288.2620484104989),
(186.69355393416936, 288.2620484104989),
(186.69355393416936, 293.1351389557999),
(173.19763437266312, 293.1351389557999)],
'/page/0/Char/366': [(171.698087754718, 288.2620484104989),
(184.81912066173794, 288.2620484104989),
(184.81912066173794, 293.1351389557999),
(171.698087754718, 293.1351389557999)],
'/page/0/Char/367': [(176.94650091752598, 288.2620484104989),
(192.3168537514636, 288.2620484104989),
(192.3168537514636, 293.1351389557999),
(176.94650091752598, 293.1351389557999)]}

In [ ]:
print(a)
min([v[0][0] for k,v in a.items()])

In [ ]:
# Rescaling boxes

import copy
def rescale_chars_to_line(boxlist):
    """
    Rescales all char bboxes inside one line so they fit into the line bbox.
    
    Args:
        line_dict (dict): JSON for a line (with bbox + Char children).
    Returns:
        dict: updated line_dict with corrected char bboxes.
    """
    boxes = copy.deepcopy(boxlist)

    lx0, ly0, lx1, ly1 = (59.981864717805415,
                              287.13748905389093,
                              278.1658976288226,
                              295.38425766901577)
    line_width = lx1 - lx0
    line_height = ly1 - ly0

    # OCR min/max from chars
    cx0 = min([v[0][0] for k,v in boxes.items()])
    cy0 = min([v[0][1] for k,v in boxes.items()])
    cx1 = max([v[2][0] for k,v in boxes.items()])
    cy1 = max([v[2][1] for k,v in boxes.items()])
    print(cx0,cy0,cx1,cy1)

    ocr_width = cx1 - cx0
    ocr_height = cy1 - cy0

    def transform_coords(coords):
        x0,y0,x1,y1 = (coords[0][0],coords[0][1],coords[2][0],coords[2][1])

        x0 = x0 * ocr_width / line_width
        y0 = y0 * ocr_height / line_height
        x1 = x1 * ocr_width / line_width
        y1 = y0 * ocr_height / line_height
        return [(x0,y0),(x0,y1),(x1,y1),(x1,y0)]

    for k,val in boxes.items():
    
        boxes.update({k:transform_coords(val)})
     
    return boxes

a = rescale_chars_to_line(a)

In [ ]:
print(a['/page/0/Char/311'])

## Utilities

In [ ]:
b = renderedboxes[0][0]
c = renderedtext[0][0]
# print(c)

blanks = []

for rchar, rcoord in zip(c.items(),b.items()):
    if rchar[1] == ' ' and re.match('^/page/0/Char/(3[0-5][0-9]|36[0-7])$',rchar[0]):
        blanks.append(rcoord[1])

print(blanks)

import copy
def rescale_chars_to_line(boxlist,k):

    boxes = []

    lx0, ly0, lx1, ly1 = (59.981864717805415,
                              287.13748905389093,
                              278.1658976288226,
                              295.38425766901577)

    lmidx = (lx1+lx0)/2

    def transform_coords(coords,k):
        x0,x1 = (coords[0][0],coords[2][0])
        x_new = (x0+x1)/2
        x_new = x_new - k * (lmidx - x_new)
        return [(x_new,ly0),(x_new,ly1),(x_new,ly1),(x_new,ly0)]

    for box in boxlist:
    
        boxes.append(transform_coords(box,k))
     
    return boxes

blanks = rescale_chars_to_line(blanks,4)

In [ ]:
def clear_doc(filename,filedir):



    init_file = filedir + filename + ".pdf"
    doc = pymupdf.open(init_file)
    page = doc[0]
    for annot in page.annots():
        page.delete_annot(annot)
    doc.saveIncr()
    doc.close()
    page=None

clear_doc("page1test","./docs_to_write_on/")

In [ ]:
# Redact specific coordinates

# def redact_pdf(filename,filedir, coords):
#     init_file = filedir + filename + ".pdf"
#     doc = fitz.open(init_file)
#     for page in doc:
#         print(page.rect.width,page.rect.height)
#         page.add_redact_annot(coords, fill=(0, 0, 0))
#     doc.save("output1.pdf")
#     doc.close()


def annotate_specific_coords(filename,filedir,coords,colour):
    init_file = filedir + filename + ".pdf"
    doc = pymupdf.open(init_file)
    page = doc[0]
    annot = page.add_polygon_annot(coords)
    annot.set_colors(stroke=colour)
    annot.update()
    doc.saveIncr()
    doc.close()
    page=None


for spacetem in blanks:
    specific_annot =  spacetem
    paint = (0,0,1)
    # (0.416, 0.416, 1)

    annotate_specific_coords("page1test","./docs_to_write_on/",specific_annot,paint)

In [ ]:
lx0,ly0,lx1,ly1 = (59.981864717805415,
                              287.13748905389093,
                              278.1658976288226,
                              295.38425766901577)

lmidx = (lx1+lx0)/2



lmidpoint = [(lmidx,ly0),(lmidx,ly0),(lmidx,ly1),(lmidx,ly1)]



paint = (0,0,0)
annotate_specific_coords("page1test","./docs_to_write_on/",lmidpoint,paint)

In [ ]:
# Plan B: Eyeball it
clear_doc("page1test","./docs_to_write_on/")
lx0,ly0,lx1,ly1 = (59.981864717805415,
                              287.13748905389093,
                              278.1658976288226,
                              295.38425766901577)

htmtext = 'We propose the use of dilated filters to construct an ag-\n'

cleantext = re.sub('[\r\n]+',"",htmtext)



def cut_up_line(tex):
    
    lx0,ly0,lx1,ly1 = (59.981864717805415,
                              287.13748905389093,
                              278.1658976288226,
                              295.38425766901577)
    

    leng = lx1-lx0

    lengcut = leng / len(tex) * 1.5

    cutboxes = np.arange(lx0,lx1,lengcut).tolist() + [lx1]
    return [[(cutboxes[j],ly0),(cutboxes[j+1],ly0),(cutboxes[j+1],ly1),(cutboxes[j],ly1)] for j,x in enumerate(cutboxes) if j < len(cutboxes)-1]




cuts = cut_up_line(cleantext)

def annotate_specific_coords(filename,filedir,coords,colour):
    init_file = filedir + filename + ".pdf"
    doc = pymupdf.open(init_file)
    page = doc[0]
    annot = page.add_polygon_annot(coords)
    annot.set_colors(stroke=colour)
    annot.update()
    doc.saveIncr()
    doc.close()
    page=None


for gaps in cuts:
    specific_annot = gaps
    paint = (0,0,1)
    # (0.416, 0.416, 1)

    annotate_specific_coords("page1test","./docs_to_write_on/",specific_annot,paint)

-----

In [ ]:
# Quickly turn PDF to images

pages = convert_from_path(current_file)
for count, page in enumerate(pages):
    page.save(f'out{count}.jpg', 'JPEG')

In [ ]:
# PDF/Image pre-processing to improve Tesseract results





# img = Image.open("./testactual_page-0001.jpg")
# print(img)
# osd = pyt.image_to_osd(img,output_type="dict")
# print(osd)


img = cv2.imread("./out0.jpg")
img = cv2.resize(img,(0,0),fx=7,fy=7)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3,3), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
invert = 255 - opening

# im = img.filter(ImageFilter.MedianFilter())
# enhancer = ImageEnhance.Contrast(im)
# im = enhancer.enhance(2)
# im = im.convert('1')
# im.save('temp2.jpg')

osd = pyt.image_to_osd(invert,output_type="dict")
print(osd)

# cv2.imshow('thresh', thresh)
# cv2.imshow('opening', opening)
# cv2.imshow('invert', invert)
# cv2.waitKey()

page_text = pyt.image_to_string(invert, lang="Greek",config="--psm 1 -c tessedit_ocr_engine_mode=1")
print(f"OCR result for page 1: {page_text}")

In [ ]:
# Break PDF into one file per page (for troubleshooting marker)

import fitz  # PyMuPDF
import os


goesin = "base_test.pdf"
comesout = "split_pdf"

def split_pdf(input_pdf, output_dir):
    # Open the source PDF
    doc = fitz.open(input_pdf)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterate over all pages
    for page_num in range(len(doc)):
        # Create a new PDF for each page
        new_doc = fitz.open()
        new_doc.insert_pdf(doc, from_page=page_num, to_page=page_num)

        # Save with page number in filename (1-indexed for readability)
        output_path = os.path.join(output_dir, f"page_{page_num+1}.pdf")
        new_doc.save(output_path)
        new_doc.close()

        print(f"Saved {output_path}")

    doc.close()


# Example usage:
split_pdf(goesin, comesout)
